In [26]:
import pandas as pd
import numpy as np
import sklearn
from IPython.display import display, HTML, SVG, Image, display_html
pd.set_option('display.max_colwidth', 500)
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
</style>'''))

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [194]:
import re
file = open('SMSSpamCollection', 'r')
data = []
answers = []
for line in file:
    line = line[:-2]
    words = line.split('\t')
    data.append(words[1])
    answers.append(int(words[0] == 'spam'))
data = np.asarray(data)
answers = np.asarray(answers)
print('spam items: ', sum(answers), ', ham items: ', len(answers) - sum(answers))
ds = pd.DataFrame({'text' : data, 'is_spam' : answers})
ds.is_spam.value_counts()

spam items:  747 , ham items:  4827


0    4827
1     747
Name: is_spam, dtype: int64

In [195]:
def fill_features(data, pattern = None, range_lens = (1, 1), ignore_words = None):
    if pattern is None:
        vectorizer = CountVectorizer(ngram_range=range_lens, stop_words=ignore_words)
    else:
        vectorizer = CountVectorizer(token_pattern=pattern, ngram_range=range_lens, stop_words=ignore_words)
    features = vectorizer.fit_transform(data)
    return vectorizer, features

In [196]:
def run_regression(features, answers, reg_const = 1.0):
    cls = LogisticRegression(C=reg_const)
    res = cross_val_score(cls, features, answers, scoring='f1', cv=10)
    print('score: ', np.mean(res), '\t std: ', np.std(res))
    return cls

In [197]:
#запускаем регрессию с параметром регуляризации по умолчанию (он равен 1.0)
#получаем качество ~0.93
vectorizer, features = fill_features(data)
cls = run_regression(features, ds.is_spam)

score:  0.929816060649 	 std:  0.0171694800418


In [198]:
def run_sample(vectorizer, cls, test_data):
    sample = vectorizer.transform(test_data)
    print(cls.predict(sample))

In [199]:
#обучаем классификатор на всей выборке
cls.fit(features, ds.is_spam)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [200]:
def print_important_coefficients(vectorizer, cls, num=20):
    coef = cls.coef_[0]
    order = np.argsort(abs(coef))[::-1]
    params = vectorizer.get_feature_names()
    for o in order[:num]:
        print(coef[o], '\t', params[o])
#выведем слова с наибольшим абсолютным весом в нашем классификаторе
print_important_coefficients(vectorizer, cls)

2.18396131592 	 txt
1.80586088208 	 text
1.77272515013 	 ringtone
1.66531251618 	 reply
1.60895276342 	 call
1.57832188296 	 uk
1.5513536655 	 won
1.53148565618 	 chat
1.45878134274 	 new
1.4377818665 	 mobile
1.40714640151 	 stop
1.37208829243 	 service
1.34836003579 	 claim
1.21130484069 	 www
1.20733832264 	 88066
1.20520090428 	 150p
1.17039655555 	 free
1.1438442765 	 cost
1.12776279557 	 message
1.11562223611 	 com


In [201]:
test_data = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB", \
"FreeMsg: Txt: claim your reward of 3 hours talk time", \
"Have you visited the last lecture on physics?", \
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", \
"Only 99$"]
#предсказываем ответ для тестовых сообщений
run_sample(vectorizer, cls, test_data)
#последние 2 примера, скорее всего, спрогнозированы неверно

[1 1 0 0 0]


In [202]:
ngrams_to_try = [(2, 2), (3, 3), (1, 3)]
for range_ngram in ngrams_to_try:
    current_vectorizer, current_features = fill_features(data, range_lens=range_ngram)
    print('ngram ', range_ngram, ':')
    run_regression(current_features, ds.is_spam)
#получаем значения 0.81, 0.72, 0.92

ngram  (2, 2) :
score:  0.814184514368 	 std:  0.0224355190018
ngram  (3, 3) :
score:  0.723910938924 	 std:  0.0180503756616
ngram  (1, 3) :
score:  0.919724379882 	 std:  0.0173382780611


In [203]:
def run_naive_bayes(features, answers):
    cls = MultinomialNB()
    res = cross_val_score(cls, features, answers, scoring='f1', cv=10)
    print('score: ', np.mean(res), '\t std: ', np.std(res))
    return cls
ngrams_to_try = [(1, 1), (2, 2), (3, 3), (1, 3)]
print('naive bayes results:')
for range_ngram in ngrams_to_try:
    current_vectorizer, current_features = fill_features(data, range_lens=range_ngram)
    print('ngram ', range_ngram, ':')
    run_naive_bayes(current_features, ds.is_spam)
#получаем значения 0.92, 0.64, 0.38, 0.88

naive bayes results:
ngram  (1, 1) :
score:  0.919974840901 	 std:  0.0135227839017
ngram  (2, 2) :
score:  0.640145169909 	 std:  0.0207635617755
ngram  (3, 3) :
score:  0.375578625938 	 std:  0.00939418934033
ngram  (1, 3) :
score:  0.88395028567 	 std:  0.017013090273


In [204]:
tfidf = TfidfVectorizer()
features_tfidf = tfidf.fit_transform(data)
print('tf*idf results:')
run_regression(features_tfidf, ds.is_spam)
#результат с использованием tf*idf хуже, чем с CountVectorizer(): 0.85 вместо 0.93

tf*idf results:
score:  0.846901921735 	 std:  0.0261167193081


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [205]:
def find_feature_by_name(vectorizer, cls, name):
    coef = cls.coef_[0]
    order = np.argsort(abs(coef))[::-1]
    params = vectorizer.get_feature_names()
    for o in order:
        if params[o] == name:
            print(name, ': ', coef[o])

In [206]:
#в спаме часто встречаются суммы денег с характерными знаками, будем учитывать их
current_vectorizer, current_features = fill_features(data, pattern=r'([A-Za-z0-9][A-Za-z0-9]+|[$]|[€]|[£])')
current_cls = run_regression(current_features, ds.is_spam)
current_cls.fit(current_features, ds.is_spam)
find_feature_by_name(current_vectorizer, current_cls, '£')
print_important_coefficients(current_vectorizer, current_cls)
find_feature_by_name(current_vectorizer, current_cls, '18')
find_feature_by_name(current_vectorizer, current_cls, '$')
#качество улучшается на ~0.007
#хотелось добавить признаки 16+ и 18+, но с запихиванием паттерна с плюсом в CountVectorizer возникли некоторые проблемы, почему-то он не распознается 
#Впрочем, оказалось, что классификатор выбрал числа 16 и 18 как признаки спама (видимо он сплиттит по плюсу)
run_sample(current_vectorizer, current_cls, test_data)
#на тестовых данных все то же самое

score:  0.934663926912 	 std:  0.0195210726626
£ :  2.77278793086
2.77278793086 	 £
1.99772912681 	 txt
1.77586882366 	 text
1.77507074611 	 ringtone
1.71392034042 	 reply
1.58753953635 	 call
1.57100383023 	 uk
1.46275751673 	 chat
1.43983490274 	 service
1.4278723663 	 stop
1.41664295377 	 new
1.37823943574 	 won
1.34313047646 	 mobile
1.28430908426 	 150p
1.21016338365 	 free
1.18114529896 	 message
1.17785096002 	 com
1.16853646463 	 www
1.14085249967 	 cost
1.11316270604 	 88066
18 :  1.03767493192
$ :  0.350598914559
[1 1 0 0 0]


In [207]:
#the - самое частовстречающееся слово в английском, причем кажется, что его отрицательный вес обусловлен только преобладанием отрицательных примеров в выборке
#пробуем убрать его из признаков
current_vectorizer, current_features = fill_features(data, ignore_words=['the'])
run_regression(current_features, ds.is_spam)
#качество незначительно ухудшается

score:  0.929784865079 	 std:  0.0180973223562


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [208]:
#попробуем теперь обработать отдельно 16+ и 18+ - заменим в данных их на слово foroldonly
def insert_forold_in_data(data):
    res_data = []
    for line in data:
        line = re.sub('16\+', r' foroldonly ', line)
        line = re.sub('18\+', r' foroldonly ', line)
        res_data.append(line)
    return np.asarray(res_data)
processed_data = insert_forold_in_data(data)
current_vectorizer, current_features = fill_features(processed_data)
current_cls = run_regression(current_features, ds.is_spam)
current_cls.fit(current_features, ds.is_spam)
find_feature_by_name(current_vectorizer, current_cls, 'foroldonly')
#качество не меняется по сравнению с вариантом по умолчанию, по-видимому все вхождения чисел 16 и 18 в датасет связаны именно с возрастными ограничениями

score:  0.929816060649 	 std:  0.0171694800418
foroldonly :  0.402337418062


In [209]:
#в спам часто входят какие-то длинные строчки из цифр - по-видимому, номера телефонов, иногда они разделены пробелами на несколько групп цифр. 
#Заменим все такие подозрительные позиции в данных на слово telephonenumber
def insert_telephone_numbers_in_data(data):
    res_data = []
    for line in data:
        line = re.sub('([0-9][ ]*[0-9][ ]*[0-9][ ]*[0-9][ ]*([0-9][ ]*)+)', r' telephonenumber ', line)
        res_data.append(line)
    return np.asarray(res_data)
processed_data = insert_telephone_numbers_in_data(data)
current_vectorizer, current_features = fill_features(processed_data, pattern=r'([A-Za-z0-9][A-Za-z0-9]+|[$]|[€]|[£])')
current_cls = run_regression(current_features, ds.is_spam)
current_cls.fit(current_features, ds.is_spam)
find_feature_by_name(current_vectorizer, current_cls, 'telephonenumber')
print(current_cls.intercept_)
print_important_coefficients(current_vectorizer, current_cls)
#качество на кросс-валидации улучшается на 0.02
#однако, новый признак имеет слишком большое влияние по сравнению с остальными, и у остальных признаков веса уменьшились по сравнению с исходным вариантом
run_sample(current_vectorizer, current_cls, test_data)
#на небольшой тестовой выборке второй пример теперь прогнозируется неправильно, видимо не хватает суммы положительных признаков из-за уменьшения их коэффициентов, 
#а самый жирным признак telephonenumber сюда не подошел

score:  0.952850166505 	 std:  0.019746294382
telephonenumber :  4.37185582468
[-4.69779586]
4.37185582468 	 telephonenumber
2.04645900537 	 £
1.61702223298 	 txt
1.59573596412 	 reply
1.49163169899 	 mobile
1.44365799387 	 uk
1.4385530076 	 ringtone
1.40573326052 	 text
1.29617980629 	 www
1.27921112096 	 tones
1.13919473872 	 content
1.13483485173 	 new
1.10508485464 	 free
1.09828217139 	 won
-1.04165039106 	 my
1.02562289005 	 chat
0.980160965074 	 order
0.975591969536 	 146tf150
0.975057743241 	 stop
-0.965375858511 	 gt
[1 0 0 0 0]


In [210]:
def get_number_of_errors(features, cls):
    ds['prediction'] = cls.predict(features)
    ds['prediction_proba'] = cls.predict_proba(features)[:,1]
    ds['confidence'] = np.abs(ds.prediction_proba - 0.5)*2
    errors = ds.loc[ds.is_spam != ds.prediction]
    return len(errors)

In [211]:
#пробуем разные параметры регуляризации
reg_consts = [0.1, 0.5, 1, 2, 5, 10, 100, 1000, 10000]
for c in reg_consts:
    print('reg_const: ', c)
    current_cls = run_regression(features, ds.is_spam, reg_const=c)
    current_cls.fit(features, ds.is_spam)
    print(get_number_of_errors(features, current_cls))
    print_important_coefficients(vectorizer, current_cls, 5)
    run_sample(vectorizer, current_cls, test_data)
#при больших c увеличивается score на кросс-валидации, но коэффициенты увеличиваются, т.к. регуляризация по сути отключается. 
#при больших c модель, обученная по всей выборке, не ошибается ни на одном элементе выборки
#Скорее всего, модель с большим c сильно переобучена - например, в ней появляются с большими коэффициентами слова, встретившиеся только в одном сообщении - для работы на новых данных эти признаки, по сути, бесполезны

reg_const:  0.1
score:  0.898796417611 	 std:  0.0193635646676
82
1.30499646563 	 txt
1.28555866483 	 call
1.03412274497 	 text
0.902327190097 	 reply
0.833659680354 	 free
[1 0 0 0 0]
reg_const:  0.5
score:  0.924664015195 	 std:  0.0163204089542
33
1.91740903899 	 txt
1.57745156337 	 text
1.51871131232 	 call
1.43011710889 	 reply
1.35501688129 	 ringtone
[1 1 0 0 0]
reg_const:  1
score:  0.929816060649 	 std:  0.0171694800418
12
2.18396131592 	 txt
1.80586088208 	 text
1.77272515013 	 ringtone
1.66531251618 	 reply
1.60895276342 	 call
[1 1 0 0 0]
reg_const:  2
score:  0.932691178624 	 std:  0.0189261733054
4
2.45580921947 	 txt
2.20126666191 	 ringtone
2.02744972204 	 text
1.90383768069 	 146tf150
1.89963515513 	 reply
[1 1 0 0 0]
reg_const:  5
score:  0.93699774907 	 std:  0.0181106684031
2
4.06541723626 	 146tf150
2.82442955557 	 txt
2.78474895019 	 ringtone
2.69139618149 	 ringtoneking
2.6913581922 	 8448
[1 1 0 0 0]
reg_const:  10
score:  0.939728510798 	 std:  0.0194246998957


In [213]:
#Выводы: неплохое качество классификации можно получить связкой CountVectorizer + LogisticRegression. Качество можно несколько улучшить, если учитывать некоторые специфичные детали спама -
#например, суммы в валюте или длинные номера. Однако в процессе важно не переобучиться, поэтому лучше обучаться сразу на нескольких коллекциях, т.к. велик риск подстроить классификатор
#под конкретную коллекцию. Для того, чтобы использовать биграммы и получить с них улучшение качества, нужны коллекции размером сильно больше, чем 5000 в нашей выборке.